In [1]:
import numpy as np
import tensorflow as tf
import scipy.io
from keras import backend as K
from keras.layers import Activation
from tensorflow.keras.utils import get_custom_objects

In [2]:
# define the filename to save the weights of the model as a .mat
filename = 'models/classify_subsytem'

# define the interval of the domain for the visible dynamics
lower_limit_x = 0
lower_limit_y = 0

upper_limit_x = 5
upper_limit_y = 5

step_size = 0.001;

# define the approximate size the small offset parameter in the smooth ReLU
gamma = 1

# define the approximate size the feedback offset parameter in the feedback motif
beta_1_and_2 = 1

# define the number of hidden species in a single hidden layer
N = 3

# define the number of epochs to use in training the model
number_of_epochs = 2

#define data
x1_train = np.arange(lower_limit_x, upper_limit_x, step_size, dtype="float32")

def band_range(x1):
    if x1 > 2 and x1 < 3:
        return 0.1
    else:
        return 1
    
y1_train = np.array([band_range(x1)for x1 in x1_train])

x_train = x1_train
y_train = y1_train

In [3]:
# define the custom activation function for the training with particular values of the relu_offset
def smooth_max_activation(x):
    return 0.5*(x + K.sqrt(K.square(x)+4*gamma))

get_custom_objects().update({'smooth_max_activation': Activation(smooth_max_activation)})

# define a neural network model that corresponds to the asymptotic neural subsytem 
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(1,), name=''),
  tf.keras.layers.Dense(N, activation='smooth_max_activation'),
  tf.keras.layers.Dense(1, activation='smooth_max_activation'),
])

# compile model with optimizer and choice of loss function
model.compile(optimizer='adam',
              loss='mse',
              metrics=['mse'])

# train the neural network model to replicate x_train -> y_train
model.fit(x_train, y_train, epochs=number_of_epochs, validation_split = 0.1, verbose=1)

Epoch 1/2
141/141 [==============================] - 1s 2ms/step - loss: 0.6563 - mse: 0.6563 - val_loss: 0.0153 - val_mse: 0.0153
Epoch 2/2
141/141 [==============================] - 0s 979us/step - loss: 0.3778 - mse: 0.3778 - val_loss: 7.6096e-04 - val_mse: 7.6096e-04


In [4]:
# view the model structure, each 'Param' corresponds to at least one rate of reaction in the chemical system
model.summary()

# save the weights of this neural network for use in ODE simulations in MATLAB
first_layer_weights = model.layers[1].get_weights()[0]
first_layer_biases = model.layers[1].get_weights()[1]
output_layer_weights = model.layers[2].get_weights()[0]
output_layer_biases =  model.layers[2].get_weights()[1]


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
  (Flatten)                  (None, 1)                 0         
                                                                 
 dense (Dense)               (None, 3)                 6         
                                                                 
 dense_1 (Dense)             (None, 1)                 4         
                                                                 
Total params: 10 (40.00 Byte)
Trainable params: 10 (40.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
print(first_layer_weights)
print(first_layer_biases)
print(output_layer_weights)
print(output_layer_biases)

[[-1.1277903 -0.8316795 -1.3108323]]
[-0.19413146 -0.1939405  -0.13648291]
[[0.71801925]
 [0.67001724]
 [0.01061856]]
[-0.22657505]


In [6]:
scipy.io.savemat(filename+'.mat', {'first_layer_weights':first_layer_weights, 
                                   'first_layer_biases':first_layer_biases, 
                                   'output_layer_weights':output_layer_weights,
                                   'output_layer_biases':output_layer_biases,
                                   'gamma': gamma,
                                  })